In [1]:
import warnings
warnings.filterwarnings('ignore')

import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import torchvision
import cv2
from ZeroShotDataset import ZeroShotDataset
from params import *

from transformers import CLIPProcessor, CLIPModel

from ConditionedSegFormerPE import ConditionedSegFormer
from LossFunc import *
import pytorch_lightning as pl
import torchmetrics as tm

In [2]:
class CLIPConditionedSegFormer(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.clip = CLIPModel.from_pretrained('openai/clip-vit-base-patch16')
        self.processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch16')

        self.segformer = ConditionedSegFormer(
            ModelParams.INCHANNELS,
            ModelParams.WIDTHS,
            ModelParams.DEPTHS,
            512,
            768,
            ModelParams.PATCH_SIZES,
            ModelParams.OVERLAP_SIZES,
            ModelParams.NUM_HEADS,
            ModelParams.EXPANSION_FACTORS,
            ModelParams.DECODER_CHANNELS,
            ModelParams.SCALE_FACTORS
        )

        self.plot_every = TrainParams.PLOT_EVERY

        self.neloss = NELoss(LossParams.ALPHA, LossParams.BETA)

        self.acc = tm.Accuracy(task="binary", threshold=LossParams.THRESHOLD)
        self.dice = DiceLoss()
        self.iou = IoULoss(LossParams.THRESHOLD)
        self.f1score = tm.F1Score(task="binary", threshold=LossParams.THRESHOLD)

        # freeze CLIP
        for param in self.clip.parameters():
            param.requires_grad = False

    def forward(self, x, condition):
        condition = self.clip.text_model(condition).last_hidden_state
        pe = self.clip.vision_model(x).last_hidden_state

        return self.segformer(x, pe, condition)
    
    def training_step(self, batch, batch_idx):
        x, condition, y = batch
        y_hat = self(x, condition)
        loss = self.neloss(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)
        
        if self.global_step % self.plot_every == 0: 
            y = y.repeat(1, 3, 1, 1)
            y_hat = y_hat.repeat(1, 3, 1, 1)

            # x_grid = torchvision.utils.make_grid(train_dataset.image_inverse_transform(x))
            # self.logger.experiment.add_image('train_sample_image', x_grid, self.global_step)

            grid = torchvision.utils.make_grid(torch.cat([y, y_hat], dim=0))
            self.logger.experiment.add_image('train_sample_mask', grid, self.global_step)

        return loss
    
    def validation_step(self, batch, batch_idx):
        x, condition, y = batch
        y_hat = self(x, condition)
        loss = self.neloss(y_hat, y)
        self.log("val_loss", loss, prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, condition, y = batch
        y_hat = self(x, condition)

        t_y_hat = torch.where(y_hat > 0.5, 1, 0).long()
        t_y = torch.where(y > 0.5, 1, 0).long()

        acc = self.acc(t_y_hat, t_y)
        dice = self.dice(y_hat, y)
        iou = self.iou(y_hat, y)
        f1 = self.f1score(t_y_hat, t_y)

        self.log("test_acc", acc, prog_bar=True)
        self.log("test_dice", dice, prog_bar=True)
        self.log("test_iou", iou, prog_bar=True)
        self.log("test_f1", f1, prog_bar=True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-7)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2000, gamma=0.99)
        return {
            "optimizer": optimizer, 
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step",
                "frequency": 1
            }         
        }

In [3]:
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
clip_processor = CLIPProcessor.from_pretrained('openai/clip-vit-base-patch16')

In [4]:
train_dataset = ZeroShotDataset(
    df = train_df, 
    # image_folder = 'C:/Datasets/COCO/P/ProcessedDataset/images/train/',
    image_folder = 'C:/Users/david/OneDrive/Documents/GitHub/TextualSegFormer/ProcessedDataset/images/train/',
    # mask_folder = "C:/Datasets/COCO/P/ProcessedDataset/masks/train/",
    mask_folder = 'C:/Users/david/OneDrive/Documents/GitHub/TextualSegFormer/ProcessedDataset/masks/train/',
    mask_size = 56,
    templates = TrainParams.TEMPLATES, 
    unseen_clases = [], 
    image_processor = clip_processor, 
    tokenizer = clip_processor.tokenizer, 
    filter_unseen = False,
    filter_seen = True
)

val_dataset = ZeroShotDataset(
    df = val_df,
    # image_folder = 'C:/Datasets/COCO/P/ProcessedDataset/images/val/',
    image_folder = 'C:/Users/david/OneDrive/Documents/GitHub/TextualSegFormer/ProcessedDataset/images/val/',
    # mask_folder = "C:/Datasets/COCO/P/ProcessedDataset/masks/val/",
    mask_folder = 'C:/Users/david/OneDrive/Documents/GitHub/TextualSegFormer/ProcessedDataset/masks/val/',
    mask_size = 56,
    templates = TrainParams.TEMPLATES,
    unseen_clases = [],
    image_processor = clip_processor,
    tokenizer = clip_processor.tokenizer,
    filter_unseen = False,
    filter_seen = True
)

In [5]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TrainParams.BATCH_SIZE, shuffle=True, collate_fn=train_dataset.collate_fn, num_workers=TrainParams.NUM_WORKERS)
val_loader = torch.utils.data.DataLoader(train_dataset, batch_size=TrainParams.BATCH_SIZE, shuffle=True, collate_fn=train_dataset.collate_fn, num_workers=TrainParams.NUM_WORKERS)

In [6]:
test_model = CLIPConditionedSegFormer()

In [7]:
# torch.set_float32_matmul_precision('medium')

In [8]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    dirpath='./checkpoints',
    filename='{epoch}-{val_loss:.2f}-{val_r2:.2f}',
    save_top_k=1,
    monitor='val_loss',
    every_n_epochs=1,
    mode='min'
)

trainer = pl.Trainer(
    accelerator='gpu',
    max_epochs=30,
    benchmark=True,
    # deterministic=True,
    precision=16,
    accumulate_grad_batches=6,
    callbacks=[
        checkpoint_callback,
        pl.callbacks.LearningRateMonitor(logging_interval='step')
    ]
)

Using 16bit None Automatic Mixed Precision (AMP)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [9]:
trainer.fit(test_model, train_loader, val_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                 | Params
---------------------------------------------------
0 | clip      | CLIPModel            | 149 M 
1 | segformer | ConditionedSegFormer | 18.5 M
2 | neloss    | NELoss               | 0     
3 | acc       | Accuracy             | 0     
4 | dice      | DiceLoss             | 0     
5 | iou       | IoULoss              | 0     
6 | f1score   | F1Score              | 0     
---------------------------------------------------
18.5 M    Trainable params
149 M     Non-trainable params
168 M     Total params
336.278   Total estimated model params s

Sanity Checking: 0it [00:00, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.fit(test_model, train_loader, val_loader)                                            │
│   2                                                                                              │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\trainer.py:608 in fit          │
│                                                                                                  │
│    605 │   │   """                                                                               │
│    606 │   │   model = self._maybe_unwrap_optimized(model)                                       │
│    607 │   │   self.strategy._lightning_module = model                                           │
│ ❱  608 │   │   call._call_and_handle_interrupt(                                                  │
│    609 │   │   │   self, self._fit_impl, model, train_dataloaders, val_dataloaders, datamodule,  │
│    610 │   │   )                                                                                 │
│    611                                                                                           │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\call.py:38 in                  │
│ _call_and_handle_interrupt                                                                       │
│                                                                                                  │
│   35 │   │   if trainer.strategy.launcher is not None:                                           │
│   36 │   │   │   return trainer.strategy.launcher.launch(trainer_fn, *args, trainer=trainer,     │
│   37 │   │   else:                                                                               │
│ ❱ 38 │   │   │   return trainer_fn(*args, **kwargs)                                              │
│   39 │                                                                                           │
│   40 │   except _TunerExitException:                                                             │
│   41 │   │   trainer._call_teardown_hook()                                                       │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\trainer.py:650 in _fit_impl    │
│                                                                                                  │
│    647 │   │   │   model_provided=True,                                                          │
│    648 │   │   │   model_connected=self.lightning_module is not None,                            │
│    649 │   │   )                                                                                 │
│ ❱  650 │   │   self._run(model, ckpt_path=self.ckpt_path)                                        │
│    651 │   │                                                                                     │
│    652 │   │   assert self.state.stopped                                                         │
│    653 │   │   self.training = False                                                             │
│                                                                                                  │
│ e:\miniconda3\envs\AI\lib\site-packages\pytorch_lightning\trainer\trainer.py:1112 in _run        │
│                                                                                                  │
│   1109 │   │                                               